<a href="https://colab.research.google.com/github/vhrique/anne2024/blob/main/02_Aprendizagem_Supervisionada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pacotes Utilizados nesta Aula

In [ ]:
import torch

# Paradigmas de Aprendizagem

![Mapa Mental](https://raw.githubusercontent.com/vhrique/anne2024/main/figures/mapa_mental.drawio.svg)
[Link para imagem em tela cheia](https://raw.githubusercontent.com/vhrique/anne2024/main/figures/mapa_mental.drawio.svg)

## Representação

## Otimização

## Avaliação

# Aprendizagem Supervisionada

## Problemas de Classificação

### Classificação Binária

### Classificação Multiclasse

### Classificação Multilabel

## Problemas de Regressão

# Algoritmos de Otimização

## Gradiente Descendente

## Momentum

## RMSProp

## ADAM

# Loss Functions

## Cross-Entropy

## Margin (Hinge)

## Mean Squared Error

## Huber